<img src="..\..\..\pics\panda2.jpg" width=600/>

<h2>Why use the spatially enabled dataframe</h2>

<ul>
  <li>In Memory</li>
  <li>Fast computation</li>
  <li>On the fly indexing</li>
  <li>Multi-platform</li>
  <li>All the benefits of pandas and more</li>
    
</ul>  

In [3]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis import GIS

In [4]:
gis = GIS('https://arcgis.com', 'bhammersley_tech')

Enter password: ········


<h1> Data Wrangling </h1>

In [5]:
df = pd.read_csv(r'C:\Users\bhammersley\OneDrive - ESRI (UK) Ltd\Documents\Presentations\Berlin_2019\data\HH.csv')
df.head()

,ID,type,Price,Bedrooms,Name,Number,Street,Town,County,Postcode
0,1,Bungalow,100000,1,NaN,10.0,Cherry Orchard,Cambridge,Cambridgeshire,CB243AY
1,2,bungalow,500000,3,Orchard Farm,NaN,Fowlmere Rd,Cambridge,Cambridgeshire,CB22 6RT
2,3,Attached,800000,6,NaN,8.0,Cherry Orchard,Cambridge,Cambridgeshire,CB243AY
3,4,semi-detached,700000,5,The Old Bakery,NaN,The Green,Royston,Cambridgeshire,SG8 7QZ
4,5,house,200000,2,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def f():
    pass

df["full_address"] = df.apply(
                              lambda row: '%s %s %s %s %s' % 
                              (row['Name'],row['Street'],row['Town'],row['County'],row['Postcode']) 
                              if str(row["Name"]) != 'nan' 
    
                              else ('%s %s %s %s %s' % 
                                    (int(row['Number']),row['Street'],row['Town'],row['County'],row['Postcode']) 
                                    if str(row["Number"]) != 'nan' else f()), axis=1)

df = df.drop(['Name', 'Number', 'Street', 'Town', 'County', 'Postcode'], axis=1).dropna().reset_index(drop=True)
df.head()

,ID,type,Price,Bedrooms,full_address
0,1,Bungalow,100000,1,10 Cherry Orchard Cambridge Cambridgeshire CB243AY
1,2,bungalow,500000,3,Orchard Farm Fowlmere Rd Cambridge Cambridgeshire CB22 6RT
2,3,Attached,800000,6,8 Cherry Orchard Cambridge Cambridgeshire CB243AY
3,4,semi-detached,700000,5,The Old Bakery The Green Royston Cambridgeshire SG8 7QZ
4,6,bungalow,100000,2,4 LONG ROAD CAMBRIDGE CAMBRIDGESHIRE CB2 8PX


In [7]:
df.type.value_counts()

bungalow         85
Attached         26
Detached         26
detached         24
attached         22
semi-detached     7
semi detached     4
Bungalow          1
Name: type, dtype: int64

In [8]:
df.type = df.type.str.lower()
q = df.type == 'semi-detached'
df.loc[q, 'type'] = 'semi detached'
df.type.value_counts()

bungalow         86
detached         50
attached         48
semi detached    11
Name: type, dtype: int64

In [11]:
sdf = pd.DataFrame.spatial.from_df(df, 'full_address')
sdf.head()

,ID,type,Price,Bedrooms,full_address,SHAPE
0,1,bungalow,100000,1,10 Cherry Orchard Cambridge Cambridgeshire CB243AY,"{""x"": 0.06696890400002076, ""y"": 52.260357174000035, ""spatialReference"": {""wkid"": 4326, ""latestWkid"": 4326}}"
1,2,bungalow,500000,3,Orchard Farm Fowlmere Rd Cambridge Cambridgeshire CB22 6RT,"{""x"": 0.06606553200003873, ""y"": 52.113959899000065, ""spatialReference"": {""wkid"": 4326, ""latestWkid"": 4326}}"
2,3,attached,800000,6,8 Cherry Orchard Cambridge Cambridgeshire CB243AY,"{""x"": 0.06707382600006895, ""y"": 52.26028190400007, ""spatialReference"": {""wkid"": 4326, ""latestWkid"": 4326}}"
3,4,semi detached,700000,5,The Old Bakery The Green Royston Cambridgeshire SG8 7QZ,"{""x"": 0.09535150000004933, ""y"": 52.10040300000003, ""spatialReference"": {""wkid"": 4326, ""latestWkid"": 4326}}"
4,6,bungalow,100000,2,4 LONG ROAD CAMBRIDGE CAMBRIDGESHIRE CB2 8PX,"{""x"": 0.14184907900005328, ""y"": 52.17872315800008, ""spatialReference"": {""wkid"": 4326, ""latestWkid"": 4326}}"


<h1> Visualisation </h1>

<ul>
  <li>Matplotlib syntax</li>    
</ul>  

<img src="..\..\..\pics\VIZ.jpg" width=600/>

In [9]:
m = gis.map()
m.center = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},'x': 17740.16697718523,'y': 6833320.7425390035}
m.zoom = 11
m

MapView(layout=Layout(height='400px', width='100%'), zoom=11.0)

In [12]:
sdf.spatial.plot(map_widget=m,
                renderer_type='u', # specify the unique value renderer using its notation 'u'
                col='type')  # column to get unique values from

True

In [13]:
bbox = sdf.spatial.bbox

In [14]:
m.draw(shape = bbox)

In [47]:
sdf.spatial.to_featurelayer('Berlin dev summit houses', gis=gis)

<Item title:"Berlin dev summit houses" type:Feature Layer Collection owner:bhammersley_tech>

<h1> Geoenrichment </h1>

In [48]:
housing_item = gis.content.get('b92ce1f0a169498f8243056a88564125')
housing_lyr = housing_item.layers[0]

In [49]:
sdf = pd.DataFrame.spatial.from_layer(housing_lyr)
sdf.head()

,Bedrooms,ID,OBJECTID,Price,SHAPE,full_address,type
0,1,1,1,100000,"{""x"": 7454.944292265932, ""y"": 6847338.967730805, ""spatialReference"": {""wkid"": 102100, ""latestWkid"": 3857}}",10 Cherry Orchard Cambridge Cambridgeshire CB243AY,bungalow
1,3,2,2,500000,"{""x"": 7354.381381231032, ""y"": 6820757.152281819, ""spatialReference"": {""wkid"": 102100, ""latestWkid"": 3857}}",Orchard Farm Fowlmere Rd Cambridge Cambridgeshire CB22 6RT,bungalow
2,6,3,3,800000,"{""x"": 7466.6241558843085, ""y"": 6847325.278199296, ""spatialReference"": {""wkid"": 102100, ""latestWkid"": 3857}}",8 Cherry Orchard Cambridge Cambridgeshire CB243AY,attached
3,5,4,4,700000,"{""x"": 10614.480426380316, ""y"": 6818300.00348284, ""spatialReference"": {""wkid"": 102100, ""latestWkid"": 3857}}",The Old Bakery The Green Royston Cambridgeshire SG8 7QZ,semi detached
4,2,6,5,100000,"{""x"": 15790.567243780766, ""y"": 6832505.628641153, ""spatialReference"": {""wkid"": 102100, ""latestWkid"": 3857}}",4 LONG ROAD CAMBRIDGE CAMBRIDGESHIRE CB2 8PX,bungalow


In [50]:
sdf.spatial.sr

{'latestWkid': 3857, 'wkid': 102100}

In [51]:
sdf.spatial.project(4326)

True

In [52]:
analysis_variables = [
    'EDUC01_CY', # 2017 POP age 16+ by Education: No qualifications
    'EDUC02_CY', # 2017 POP age 16+ by Education: Level 1 qualifications
    'EDUC03_CY', # 2017 POP age 16+ by Education: Level 2 qualifications
    'EDUC04_CY', # 2017 POP age 16+ by Education: Level 3 qualifications
    'EDUC05_CY', # 2017 POP age 16+ by Education: Level 4 qualifications and above
    
    'UNEMP_CY', # 2016 unemployed population
    
    'POPDENS_CY', # 2017 population density
    
    'PPPC_CY', # 2017 Purchasing power by capita
    
    'HINC01_CY', # Total households in lowest quintile (below £19,158)
    'HINC01_CY', # Total households in 2nd quintile (£19,158 £28,123)
    'HINC01_CY', # Total households in 3rd quintile (£28,124 to £38,084)
    'HINC01_CY', # Total households in 4th quintile (£38,085 to £54,646)
    'HINC01_CY', # Total households in 5th quintile (£54,646 to £19,158)
    
    'HTYP01A_CY', # Households by type: Single person
    'HTYP02A_CY', # Households by type: Married couple with dependent children
    'HTYP03A_CY', # Households by type: Married couple with no children
    'HTYP04A_CY', # Households by type: Cohabiting couple with dependent children
    'HTYP05A_CY', # Households by type: Cohabiting couple with no dependent children
    'HTYP06A_CY', # Households by type: Single parent with dependent children
    'HTYP07A_CY', # Households by type: Single parent with no dependent children
    'HTYP08A_CY' # Households by type: Other household types
    
]


In [53]:
from arcgis.geoenrichment import enrich

enriched_sdf = enrich(sdf, analysis_variables=analysis_variables)

In [54]:
enriched_sdf.head()

,Bedrooms,EDUC01_CY,EDUC02_CY,EDUC03_CY,EDUC04_CY,EDUC05_CY,HINC01_CY,HTYP01A_CY,HTYP02A_CY,HTYP03A_CY,...,aggregationMethod,apportionmentConfidence,areaType,bufferRadii,bufferUnits,bufferUnitsAlias,full_address,populationToPolygonSizeRating,sourceCountry,type
0,1,195,136,169,165,541,97,165,131,224,...,BlockApportionment:GB.CensusAreas,2.219,RingBuffer,1,esriMiles,Miles,10 Cherry Orchard Cambridge Cambridgeshire CB243AY,1.539,GB,bungalow
1,3,149,111,133,110,440,70,97,119,190,...,BlockApportionment:GB.CensusAreas,2.219,RingBuffer,1,esriMiles,Miles,Orchard Farm Fowlmere Rd Cambridge Cambridgeshire CB22 6RT,1.539,GB,bungalow
2,6,195,136,169,165,541,97,165,131,225,...,BlockApportionment:GB.CensusAreas,2.219,RingBuffer,1,esriMiles,Miles,8 Cherry Orchard Cambridge Cambridgeshire CB243AY,1.539,GB,attached
3,5,93,73,82,90,359,50,92,84,132,...,BlockApportionment:GB.CensusAreas,2.219,RingBuffer,1,esriMiles,Miles,The Old Bakery The Green Royston Cambridgeshire SG8 7QZ,1.539,GB,semi detached
4,2,1715,991,1359,1522,7474,1064,2601,1299,1733,...,BlockApportionment:GB.CensusAreas,2.219,RingBuffer,1,esriMiles,Miles,4 LONG ROAD CAMBRIDGE CAMBRIDGESHIRE CB2 8PX,1.539,GB,bungalow


In [55]:
enriched_sdf.spatial.to_featurelayer('enriched houses')

<Item title:"enriched houses" type:Feature Layer Collection owner:bhammersley_tech>